<a href="https://colab.research.google.com/github/Somesh140/iNeauron_FSDS_Assignments/blob/main/DL_CV_practical/DL_CV_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import os
import tarfile
import urllib.request
import numpy as np
from PIL import Image
from flask import Flask, render_template, request

In [2]:
# Download and extract CIFAR-100 dataset
dataset_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
dataset_file = "cifar-10-python.tar.gz"
dataset_folder = "cifar-10-python"
if not os.path.exists(dataset_folder):
    urllib.request.urlretrieve(dataset_url, dataset_file)
    with tarfile.open(dataset_file, "r:gz") as tar:
        tar.extractall()
    os.remove(dataset_file)

In [3]:
# Load CIFAR-100 dataset
def load_dataset():
    def unpickle(file):
        import pickle
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict

    train_data = []
    train_labels = []
    test_data = []
    test_labels = []

    # Load training data
    for i in range(1, 6):
        file_path = os.path.join(dataset_folder, "train_data_batch_{}".format(i))
        data = unpickle(file_path)
        train_data.append(data[b'data'])
        train_labels += data[b'fine_labels']

    # Load test data
    file_path = os.path.join(dataset_folder, "test_data")
    data = unpickle(file_path)
    test_data.append(data[b'data'])
    test_labels += data[b'fine_labels']

    # Convert data to numpy arrays
    train_data = np.concatenate(train_data, axis=0)
    test_data = np.concatenate(test_data, axis=0)
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)

    return train_data, train_labels, test_data, test_labels

In [5]:

# Build and train your model here using TensorFlow or PyTorch
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Load CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Preprocess and normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to categorical
y_train = to_categorical(y_train, num_classes=100)
y_test = to_categorical(y_test, num_classes=100)

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test))

169001437/169001437 [==============================] - 6s 0us/step
Epoch 1/20
391/391 [==============================] - 352s 895ms/step - loss: 4.0397 - accuracy: 0.0822 - val_loss: 3.4214 - val_accuracy: 0.1917
Epoch 2/20
391/391 [==============================] - 356s 911ms/step - loss: 3.3187 - accuracy: 0.2020 - val_loss: 2.8901 - val_accuracy: 0.2940
Epoch 3/20
391/391 [==============================] - 353s 903ms/step - loss: 2.9396 - accuracy: 0.2741 - val_loss: 2.6354 - val_accuracy: 0.3402
Epoch 4/20
391/391 [==============================] - 353s 903ms/step - loss: 2.7187 - accuracy: 0.3124 - val_loss: 2.5121 - val_accuracy: 0.3682
Epoch 5/20
391/391 [==============================] - 354s 905ms/step - loss: 2.5509 - accuracy: 0.3500 - val_loss: 2.3485 - val_accuracy: 0.4012
Epoch 6/20
391/391 [==============================] - 352s 900ms/step - loss: 2.4136 - accuracy: 0.3757 - val_loss: 2.2969 - val_accuracy: 0.4120
Epoch 7/20
391/391 [==============================] - 358

In [1]:
# Save the model
model.save('cifar100_model.h5')

NameError: ignored

In [ ]:

# Create Flask application
app = Flask(__name__)

# Define route for home page
@app.route('/')
def home():
    return render_template('index.html')

# Define route for image classification
@app.route('/classify', methods=['POST'])
def classify():
    # Get uploaded image from request
    uploaded_image = request.files['image']
    
    # Save the uploaded image to a temporary file
    temp_image_path = 'temp_image.jpg'
    uploaded_image.save(temp_image_path)
    
    # Open and preprocess the image
    image = Image.open(temp_image_path)
    image = image.resize((32, 32))  # Resize the image to match CIFAR-100 size
    image = np.array(image)
    image = image.astype('float32') / 255.0
    
    # Perform image classification using your trained model
    # Replace this with your own code for prediction
    predicted_class = 0
    prediction_score = 0.0
    
    return render_template('result.html', predicted_class=predicted_class, prediction_score=prediction_score)

# Run the Flask application
if __name__ == '__main__':
    app.run(debug=True)
